<a href="https://colab.research.google.com/github/yamanoyu/atcorder/blob/master/Educational_DP_Contest_DP_%E3%81%BE%E3%81%A8%E3%82%81%E3%82%B3%E3%83%B3%E3%83%86%E3%82%B9%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A - Frog 1

In [7]:
n = int(input())
h_list = list(map(int, input().split()))

INF = float('inf')

dp = [INF] * n
dp[0] = 0

for i in range(n):
  if i == 0:
    continue
  elif i == 1:
    dp[i] = abs(h_list[i] - h_list[i - 1])
  else:
    dp[i] = min(dp[i - 1] + abs(h_list[i] - h_list[i - 1]), dp[i - 2] + abs(h_list[i] - h_list[i - 2]))

ans = dp[-1]
print(ans)

6
30 10 60 10 60 50
40


## B - Frog 2

In [23]:
n, k = map(int, input().split())
h_list = list(map(int, input().split()))

INF = float('inf')

dp = [INF] * n
dp[0] = 0

for i in range(n):
  if i == 0:
    continue
  else:
    for j in range(0, min(i + 1, k + 1)):
      dp[i] = min(dp[i], dp[i - j] + abs(h_list[i] - h_list[i - j]))

ans = dp[-1]
print(ans)

10 4
40 10 20 70 80 10 20 70 80 60
40


## C - Vacation

In [ ]:
n = int(input())
abc_list = [list(map(int, input().split())) for _ in range(n)]

dp = [[0, 0, 0] for _ in range(n)]
dp[0] = abc_list[0]

for i in range(n):
  if i == 0:
    continue
  else:
    dp[i][0] = max(dp[i - 1][1], dp[i - 1][2]) + abc_list[i][0]
    dp[i][1] = max(dp[i - 1][0], dp[i - 1][2]) + abc_list[i][1]
    dp[i][2] = max(dp[i - 1][0], dp[i - 1][1]) + abc_list[i][2]

ans = max(dp[-1])
print(ans)

## D - Knapsack 1

In [ ]:
n, w = map(int, input().split())
wv_list = [list(map(int, input().split())) for _ in range(n)]

dp = [[0] * (w + 1) for _ in range(n)]

for i in range(n):
  dp[i] = dp[i - 1].copy()
  _dp = dp[i - 1].copy()
  
  for j in range(w + 1):
    if j + wv_list[i][0] > w:
      break
    
    dp[i][j + wv_list[i][0]] = max(_dp[j + wv_list[i][0]], _dp[j] + wv_list[i][1])

ans = max(dp[-1])
print(ans)

## E - Knapsack 2

In [5]:
n, w = map(int, input().split())
wv_list = [list(map(int, input().split())) for _ in range(n)]

INF = float('inf')
v_max = sum([v for _, v in wv_list]) + 1

dp = [INF] * v_max
dp[0] = 0

for w_l, v_l in wv_list:
  _dp = dp.copy()
  for j in range(v_max):
    if _dp[j] != INF:
      if _dp[j] + w_l <= w:
        dp[j + v_l] = min(_dp[j] + w_l, _dp[j + v_l])
        
ans = 0
for dp_i, dp_v in enumerate(dp):
  if dp_v <= w:
    ans = dp_i
    
print(ans)

3 1
22 2
2 2
2 2
0


## F - LCS

In [ ]:
# TLE

s = input()
t = input()
s_len = len(s)
t_len = len(t)

dp = [[''] * s_len for _ in range(t_len)]

if s[0] == t[0]:
  dp[0][0] = s[0]

for i in range(1, s_len):
  if s[i] == t[0]:
    dp[0][i] += s[i]
  else:
    dp[0][i] = dp[0][i - 1]

for j in range(1, t_len):
  if t[j] == s[0]:
    dp[j][0] += t[j]
  else:
    dp[j][0] = dp[j - 1][0]

for i in range(1, t_len):
  for j in range(1, s_len):
    if t[i] == s[j]:
      dp[i][j] = dp[i - 1][j - 1] + t[i]
    else:
      dp[i][j] = dp[i - 1][j] if len(dp[i - 1][j]) > len(dp[i][j - 1]) else dp[i][j - 1]

ans = dp[-1][-1]
print(ans)

## G - Longest Path

In [ ]:
from collections import defaultdict, deque
from typing import List


n, m = map(int, input().split())
graph = []
for _ in range(m):
  x, y = map(int, input().split())
  graph.append([x - 1, y - 1])


dp = [0] * n
visited_list = [False] * n


# トポロジカルソートクラス
class tpsort():
  def __init__(self, nodes, edges) -> None:
    self.nodes = nodes
    self.edges = edges
    self.graph = {}
    self.nowgraph = {}
    self.first_readies = []
    self.readies = []
    
  def sort(self) -> None:
    # nodesとedgesに対してトポロジカルソート実行 -> self.graphに格納
    outs = defaultdict(list)
    ins = defaultdict(int)
    for u, v in self.edges:
      outs[u].append(v)
      ins[v] += 1
    q = deque(u for u in self.nodes if ins[u] == 0)
    self.readies = list(q.copy())
    self.first_readies = list(q.copy())
    insc = ins.copy()
    while q:
      u = q.popleft()
      self.graph[u] = {
          'status':0,
          'next':outs[u] if outs[u] else [],
          'ins':insc[u]
          }
      for v in outs[u]:
        ins[v] -= 1
        if ins[v] == 0:
          q.append(v)
    self.nowgraph = self.graph.copy()

  def done(self, node) -> bool:
    # 入力したnodeを完了にする。
    if node in self.get_ready():
      # 入力ノードが実行可能の場合，status=1
      self.graph[node]['status'] = 1
      # nowgraphの連結ノードからinsを-1
      for v in self.nowgraph[node]['next']:
        self.nowgraph[v]['ins'] -= 1
        if self.nowgraph[v]['ins'] == 0:
          self.readies.append(v)
      # nowgraphとreadiesからnodeを削除
      self.nowgraph.pop(node)
      self.readies.remove(node)
      return True
    else:
      # 入力nodeが実行不可の場合
      return False

  def get_ready(self) -> List:
    # 現時点で実行可能なnodeのリストを返す
    return self.readies

  def get_undone(self) -> List:
    # 現時点で未実行なnodeのリストを返す
    return [i for i in self.graph if self.graph[i]['status']==0]
    
  def reset_done(self) -> None:
    # 全てundoneに戻す
    self.readies = list(self.first_readies.copy())
    for u in self.nodes:
      self.graph[u]['status'] = 0
    self.nowgraph = self.graph.copy()

  def is_complete(self) -> bool:
    # すべて完了していたらTrue, 1つ以上未実行があればFalse
    return True if not self.get_undone() else False


# 頂点をトポロジカルソート(関数で実装する)
nodes = list(range(n))
tps = tpsort(nodes, graph)
tps.sort()

for i in tps.graph:
  for j in tps.graph[i]['next']:
    dp[j] = max(dp[j], dp[i] + 1)

ans = max(dp)

print(ans)

## H - Grid 1

In [ ]:
h, w = map(int, input().split())
a_list = [list(input()) for _ in range(h)]

MOD = 10 ** 9 + 7

dp = [[0] * w for _ in range(h)]
dp[0][0] = 1

for i in range(h):
  for j in range(w):
    if i == 0 and j == 0:
      continue
    elif i == 0:
      if a_list[i][j] == '.':
        dp[i][j] = dp[i][j - 1]
        continue
    elif j == 0:
      if a_list[i][j] == '.':
        dp[i][j] = dp[i - 1][j]
        continue
    else:
      if a_list[i][j] == '#':
        continue
      if a_list[i - 1][j] == '#' and a_list[i][j - 1] == '#':
        continue
      elif a_list[i - 1][j] == '#':
        dp[i][j] = dp[i][j - 1]
      elif a_list[i][j - 1] == '#':
        dp[i][j] = dp[i - 1][j]
      else:
        dp[i][j] = (dp[i][j - 1] + dp[i - 1][j]) % MOD

ans = dp[-1][-1]
print(ans)

## I - Coins

In [16]:
import math


n = int(input())
p_list = list(map(float, input().split()))

dp = [[0] * (n + 1) for _ in range(n)]
dp[0][0] = 1 - p_list[0]
dp[0][1] = p_list[0]

for i in range(1, n):
  for j in range(i + 2):
    if j == 0:
      dp[i][j] = dp[i - 1][j] * (1 - p_list[i])
    else:
      dp[i][j] = dp[i - 1][j] * (1 - p_list[i]) + dp[i - 1][j - 1] * p_list[i]

# ans = sum(dp[-1][- (int(math.ceil(n / 2))) :])
ans = sum(dp[-1][n // 2 + 1:])
print(ans)

5
0.42 0.01 0.42 0.99 0.42
0.3821815872


## J - Sushi

In [1]:
n = int(input())
a_list = list(map(int, input().split()))

single_num = 0
duble_num = 0
triple_num = 0

for a in a_list:
  if a == 1:
    single_num += 1
  elif a == 2:
    duble_num += 1
  else:
    triple_num += 1


dp = [[[0] * (n + 2) for _ in range(n + 2)] for _ in range(n + 2)]

# dp[1][0][0] = n
# dp[0][1][0] = n * 2
# dp[0][0][1] = n * 3

for k in range(n + 1):
  for j in range(n + 1 - k):
    for i in range(n + 1 - j - k):
      zero_num = n - (i + j + k)
      if i + j + k == 0: continue
      init = 1 / (1 - zero_num / n)
      dp[i][j][k] = init
      if i > 0:
        dp[i][j][k] += dp[i - 1][j][k] * i / (n - zero_num)
      if j > 0:
        dp[i][j][k] += dp[i + 1][j - 1][k] * j / (n - zero_num)
      if k > 0:
        dp[i][j][k] += dp[i][j + 1][k - 1] * k / (n - zero_num)

ans = dp[single_num][duble_num][triple_num]
        
print(ans)

3
1 1 1
5.499999999999999


## K - Stones

In [13]:
n, k = map(int, input().split())
a_list = list(map(int, input().split()))

dp = [False] * k

for i in range(k):
  if dp[i]: continue
  if i == 0:
    if a_list[0] == 1: dp[i] = True
  else:
    for a in a_list:
      if (i + 1) - a >= 0:
        if (i + 1) - a < a_list[0] or dp[i - a] == False:
          dp[i] = True
          break

ans = 'First' if dp[-1] else 'Second'
print(ans)

1 100000
1
Second
